In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()



pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210617-1306-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [2]:
#Random Forrest hail table
run_hash="36fa55db"

ht = hl.read_table(
        f'{lustre_dir}/variant_qc/models/{run_hash}/rf_result_MegaWES_new.ht')

In [2]:
mt_filtered=hl.read_matrix_table(f'{lustre_dir}/variant_qc/MegaWESSanger_cohorts_AC_synonymous_filtered_june_2021.mt')

In [3]:
mt_filtered.entries().describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
        RAW_MQandDP: array<int32>, 


2021-06-17 13:06:18 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'


In [26]:
mt_trans=mt_filtered.filter_entries(mt_filtered.info.AC[0]==2)

mt_trans_count=mt_trans.group_cols_by(mt_trans.fam_id).aggregate(transmitted_singletons_count=hl.agg.count_where((mt_trans.info.AC[0] == 2) 
                                & (mt_trans.proband_entry.GT ==hl.Call([0,1])) & 
                                                                                            
                               ( ( (mt_trans.father_entry.GT ==hl.Call([0,1])) & (mt_trans.mother_entry.GT ==hl.Call([0,0])) )
                               
                                | ( (mt_trans.mother_entry.GT==hl.Call([0,1])) & (mt_trans.father_entry.GT ==hl.Call([0,0])) )  
                                
                                
                                )   ))

In [4]:
mt_trans=mt_filtered.filter_entries(mt_filtered.info.AC[0]==2)
en=mt_trans.entries()

In [7]:
en.show()

+---------------+------------+------+----------+----------+--------------+----------------+---------+----------------------+
| locus         | alleles    | rsid |     qual | filters  | info.AC      | info.AF        | info.AN | info.AS_BaseQRankSum |
+---------------+------------+------+----------+----------+--------------+----------------+---------+----------------------+
| locus<GRCh38> | array<str> | str  |  float64 | set<str> | array<int32> | array<float64> |   int32 | array<float64>       |
+---------------+------------+------+----------+----------+--------------+----------------+---------+----------------------+
| chr1:69900    | ["G","A"]  | NA   | 4.44e+01 | NA       | [2]          | [3.57e-05]     |   56088 | NA                   |
| chr1:69900    | ["G","A"]  | NA   | 4.44e+01 | NA       | [2]          | [3.57e-05]     |   56088 | NA                   |
| chr1:69900    | ["G","A"]  | NA   | 4.44e+01 | NA       | [2]          | [3.57e-05]     |   56088 | NA                   |
| chr1:69900    | ["G","A"]  | NA   | 4.44e+01 | NA       | [2]          | [3.57e-05]     |   56088 | NA                   |
| chr1:69900    | ["G","A"]  | NA   | 4.44e+01 | NA       | [2]          | [3.57e-05]     |   56088 | NA                   |
| chr1:69900    | ["G","A"]  | NA   | 4.44e+01 | NA       | [2]          | [3.57e-05]     |   56088 | NA                   |
| chr1:69900    | ["G","A"]  | NA   | 4.44e+01 | NA       | [2]          | [3.57e-05]     |   56088 | NA                   |
| chr1:69900    | ["G","A"]  | NA   | 4.44e+01 | NA       | [2]          | [3.57e-05]     |   56088 | NA                   |
| chr1:69900    | ["G","A"]  | NA   | 4.44e+01 | NA       | [2]          | [3.57e-05]     |   56088 | NA                   |
| chr1:69900    | ["G","A"]  | NA   | 4.44e+01 | NA       | [2]          | [3.57e-05]     |   56088 | NA                   |
+---------------+------------+------+----------+----------+--------------+----------------+---------+----------------------+

+----------------+-------------------------+----------------+-------------------+----------------+------------------------+----------------+
| info.AS_FS     | info.AS_InbreedingCoeff | info.AS_MQ     | info.AS_MQRankSum | info.AS_QD     | info.AS_ReadPosRankSum | info.AS_SOR    |
+----------------+-------------------------+----------------+-------------------+----------------+------------------------+----------------+
| array<float64> | array<float64>          | array<float64> | array<float64>    | array<float64> | array<float64>         | array<float64> |
+----------------+-------------------------+----------------+-------------------+----------------+------------------------+----------------+
| NA             | [-1.15e-01]             | NA             | NA                | [2.97e+01]     | NA                     | NA             |
| NA             | [-1.15e-01]             | NA             | NA                | [2.97e+01]     | NA                     | NA             |
| NA             | [-1.15e-01]             | NA             | NA                | [2.97e+01]     | NA                     | NA             |
| NA             | [-1.15e-01]             | NA             | NA                | [2.97e+01]     | NA                     | NA             |
| NA             | [-1.15e-01]             | NA             | NA                | [2.97e+01]     | NA                     | NA             |
| NA             | [-1.15e-01]             | NA             | NA                | [2.97e+01]     | NA                     | NA             |
| NA             | [-1.15e-01]             | NA             | NA                | [2.97e+01]     | NA                     | NA             |
| NA             | [-1.15e-01]             | NA             | NA                | [2.97e+01]     | NA                     | NA             |
| NA             | [-1.15e-01]             | NA             | NA                | [2.97e+01]     | NA                     | NA             |
| NA             | [-1.15

In [28]:
Total_transmitted_singletons=mt_trans_count.aggregate_entries(hl.agg.count_where(mt_trans_count.transmitted_singletons_count >0))
print(Total_transmitted_singletons)

74180


In [17]:
Total_transmitted_singletons=mt_trans_count.aggregate_entries(hl.agg.count_where(mt_trans_count.transmitted_singletons_count >0))
print(Total_transmitted_singletons)

74180


In [4]:
# Calculations based on Eugene's logic for the matrixtable, calculations are done on entries, so cannot be done on hail table. 

mt_trans=mt_filtered.filter_entries(mt_filtered.info.AC[0]==2)
mt_untrans=mt_filtered.filter_entries(mt_filtered.info.AC[0]==1)

mt_trans_count=mt_trans.group_cols_by(mt_trans.id).aggregate(transmitted_singletons_count=hl.agg.count_where((mt_trans.info.AC[0] == 2) 
                                & (mt_trans.proband_entry.GT.is_h_non_ref) & 
                                                                                            
                               (  (mt_trans.father_entry.GT.is_non_ref()) | (mt_trans.mother_entry.GT.is_non_ref() ))
                                                                                                             ))

mt_untrans_count = mt_untrans.group_cols_by(mt_untrans.id).aggregate(
    untransmitted_singletons_count=hl.agg.count_where((mt_untrans.info.AC[0] == 1) & 
                     (mt_untrans.proband_entry.GT.is_hom_ref()) & (
                     (mt_untrans.father_entry.GT.is_non_ref()) |
                     (mt_untrans.mother_entry.GT.is_het_non_ref()) )
                                                     ))

In [ ]:
Total_transmitted_singletons=mt_trans_count.aggregate_entries(hl.agg.count_where(mt_trans_count.transmitted_singletons_count >0))
print(Total_transmitted_singletons)
Total_untransmitted_singletons=mt_untrans_count.aggregate_entries(hl.agg.count_where(mt_untrans_count.untransmitted_singletons_count >0))
print(Total_untransmitted_singletons)
Ratio_transmitted_untransmitted=Total_transmitted_singletons/Total_untransmitted_singletons
print(Ratio_transmitted_untransmitted)

In [2]:
famstats_ht=hl.read_table(f'{lustre_dir}/MegaWES_family_stats_notannoted.mt')
mt=hl.read_matrix_table(f'{lustre_dir}/variant_qc/MegaWESSanger_cohorts_sampleQC_filtered_autosomes_split.mt')
mt = mt.annotate_rows(family_stats=famstats_ht[mt.row_key].family_stats)

In [7]:
htf.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'family_stats': array<struct {
        mendel: struct {
            errors: int64
        }, 
        tdt: struct {
            t: int64, 
            u: int64, 
            chi_sq: float64, 
            p_value: float64
        }, 
        unrelated_qc_callstats: struct {
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            homozygote_count: array<int32>
        }, 
        meta: dict<str, str>
    }> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [2]:
#Random Forrest hail table
run_hash="36fa55db"

ht = hl.read_table(
        f'{lustre_dir}/variant_qc/models/{run_hash}/rf_result_MegaWES_new.ht')

In [3]:
ht.count()

7515314

In [2]:
# Matrixtable
mt_trios = hl.read_matrix_table(
        f'{lustre_dir}/MegaWES_trio_table.mt')
ht_synonymous=hl.read_table( f'{lustre_dir}/grch38_synonymous_variants.ht')
mt_trios = mt_trios.annotate_rows(consequence=ht_synonymous[mt_trios.row_key].consequence)

mt_filtered = mt_trios.filter_rows((mt_trios.info.AC[0] <= 2) & (
        mt_trios.consequence == "synonymous_variant"))

#mt_filtered=hl.read_matrix_table(f'{lustre_dir}/variant_qc/MegaWESSanger_cohorts_AC_synonymous_filtered_june_2021.mt')

In [2]:

ht_stats=hl.read_table(f'{lustre_dir}/variant_qc/MegaWES_stats.ht')

In [ ]:
ht_stats.summarize()

In [3]:
mt_filtered.count()

(870093, 9742)

In [4]:
mt_filtered=hl.read_matrix_table(f'{lustre_dir}/variant_qc/MegaWESSanger_cohorts_AC_synonymous_filtered_june_2021.mt')

In [5]:
mt_filtered.count()

(102194, 9742)

In [ ]:
mt_filtered.entries().show()

2021-06-16 15:59:45 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'


In [ ]:
# Calculations based on Eugene's logic for the matrixtable, calculations are done on entries, so cannot be done on hail table. 

mt_trans=mt_filtered.filter_entries(mt_filtered.info.AC[0]==2)
mt_untrans=mt_filtered.filter_entries(mt_filtered.info.AC[0]==1)
mt_trans_count=mt_trans.group_cols_by(mt_trans.id).aggregate(transmitted_singletons_count=hl.agg.count_where((mt_trans.info.AC[0] == 2) 
                                                                                            & (hl.is_defined(mt_trans.proband_entry.GT)) & 
                                                                                            
                               ( ((mt_trans.proband_entry.GT) == mt_trans.father_entry.GT) | (mt_trans.proband_entry.GT == mt_trans.mother_entry.GT )))) 

mt_untrans_count = mt_untrans.group_cols_by(mt_untrans.id).aggregate(
    untransmitted_singletons_count=hl.agg.count_where((mt_untrans.info.AC[0] == 1) & 
                               ( (mt_untrans.proband_entry.GT == mt_untrans.father_entry.GT) | (mt_untrans.proband_entry.GT == mt_untrans.mother_entry.GT ))))
                    
                    
                                                      #&
                     #(mt_untrans.proband_entry.GT.is_hom_ref()) &
                   #  (mt_untrans.father_entry.GT.is_het_ref()) |
                    # (mt_untrans.mother_entry.GT.is_het_ref()))))

In [ ]:
hl.is_defined

In [ ]:
print(mt_untrans_count.untransmitted_singletons_count.summarize())

In [ ]:
Total_transmitted_singletons=mt_trans_count.aggregate_entries(hl.agg.count_where(mt_trans_count.transmitted_singletons_count==1))
print(Total_transmitted_singletons)
Total_untransmitted_singletons=mt_untrans_count.aggregate_entries(hl.agg.count_where(mt_untrans_count.untransmitted_singletons_count==1))
print(Total_untransmitted_singletons)
Ratio_transmitted_untransmitted=Total_transmitted_singletons/Total_untransmitted_singletons
print(Ratio_transmitted_untransmitted)


In [36]:
mt2=mt_trans_count.annotate_rows(variant_transmitted_singletons=hl.agg.count_where(mt_trans_count.transmitted_singletons_count==1))
print(mt2.variant_transmitted_singletons.summarize())

mt3=mt_untrans_count.annotate_rows(variant_untransmitted_singletons=hl.agg.count_where(mt_untrans_count.untransmitted_singletons_count==1))
print(mt3.variant_untransmitted_singletons.summarize())

Non-missing,102194 (100.00%)
Missing,0
Minimum,0
Maximum,4
Mean,0.79
Std Dev,0.46


None


Non-missing,102194 (100.00%)
Missing,0
Minimum,0
Maximum,9742
Mean,943.66
Std Dev,2859.40


None


In [37]:
#Annotate hail table with the values of the singleton counts

ht=ht.annotate(variant_transmitted_singletons=mt2.rows()[ht.key].variant_transmitted_singletons)
ht=ht.annotate(variant_untransmitted_singletons=mt3.rows()[ht.key].variant_untransmitted_singletons)

In [87]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        qual: float64, 
        AN: int32, 
        BaseQRankSum: float64, 
        DP: int32, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        SOR: float64, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'AC': array<int32> 
    'AF': array<float64> 
    'AN': int32 
    'AS_BaseQRankSum': array<floa

In [134]:
ht_stats=hl.read_table(f'{lustre_dir}/variant_qc/MegaWES_stats.ht')
ht=ht.annotate(fam=ht_stats[ht.key])

In [135]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        qual: float64, 
        AN: int32, 
        BaseQRankSum: float64, 
        DP: int32, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        SOR: float64, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'AC': array<int32> 
    'AF': array<float64> 
    'AN': int32 
    'AS_BaseQRankSum': array<floa

In [133]:
ht=ht.annotate(n_trans_singletons=hl.agg.filter(
            ht.ac_raw == 2, hl.agg.sum(ht.fam.n_transmitted_raw)
        ))

n_untrans_singletons=hl.agg.filter(
            (ht.ac_raw < 3) & (ht.ac_qc_samples_unrelated_raw == 1),
            hl.agg.sum(fam.n_untransmitted_raw),
        ),

2021-06-16 14:19:57 Hail: ERROR: 'Table.annotate: field 'n_trans_singletons'' does not support aggregation


ExpressionException: 'Table.annotate: field 'n_trans_singletons'' does not support aggregation

In [122]:
ratio=f['n_trans_singletons']/f['n_untrans_singletons']

In [124]:
hl.eval(ratio)

2021-06-16 14:12:42 Hail: ERROR: 'eval_timed' does not support aggregation


ExpressionException: 'eval_timed' does not support aggregation

In [111]:
hl.count_where((n_trans_singletons=hl.agg.filter(
            ht.ac_raw == 2, hl.agg.sum(ht.fam.n_transmitted_raw))
        )
n_untrans_singletons=hl.agg.filter(
            (ht.ac_raw < 3) & (ht.ac_qc_samples_raw == 1),
            hl.agg.sum(ht.fam.n_untransmitted_raw)))

SyntaxError: invalid syntax (<ipython-input-111-bd92775597c5>, line 1)

In [38]:
ht.variant_transmitted_singletons.summarize()

Non-missing,102194 (1.36%)
Missing,7413120 (98.64%)
Minimum,0
Maximum,4
Mean,0.79
Std Dev,0.46


In [39]:
# The problem is here:
ht.variant_untransmitted_singletons.summarize()

Non-missing,102194 (1.36%)
Missing,7413120 (98.64%)
Minimum,0
Maximum,9742
Mean,943.66
Std Dev,2859.40
